# Large Language Model Usage in Digital Discourse

As of December 2025 (things change fast!): Large Language Models (LLMs) have become a practical methodological tool in Digital Discourse Analysis, not as replacements for theory-driven interpretation, but as instruments that scale and accelerate interpretive labor.

Their primary value lies in handling discourse-level phenomena—such as framing, stance, argumentation, and narrative roles—that are difficult to capture with traditional rule-based or statistical NLP methods.

When used critically and transparently, LLMs enable researchers to annotate large corpora, explore patterns across time and languages, and support comparative analysis that would otherwise be infeasible. At the same time, LLM outputs remain probabilistic, interpretive, and error-prone, making methodological control, validation, and human oversight essential. Properly positioned, LLMs function as advanced analytical assistants within established Digital Discourse frameworks rather than as autonomous authorities on meaning.

## 1. Discourse-Level Annotation at Scale

- Automatic labeling of discourse features that are contextual and interpretive rather than purely lexical  
- Typical annotation tasks:
  - Stance detection (support / oppose / neutral)
  - Framing detection (economic, moral, security, cultural, etc.)
  - Actor roles (government, citizen, expert, institution, scapegoat)
  - Narrative positions (victim, threat, hero, authority)
  - Politeness, aggression, irony, sarcasm
  - Modality and certainty (hedging, obligation, inevitability)
- Strengths:
  - Handles multi-sentence context
  - Performs better than keyword or rule-based systems on indirect language
- Methodological notes:
  - Use closed and well-defined label sets
  - Validate on a human-annotated subset
  - Treat outputs as noisy annotations suitable for aggregation

---

## 2. Argumentation and Rhetorical Structure Extraction

- Identification of argument components within texts:
  - Claims
  - Premises
  - Conclusions
  - Implicit warrants
- Detection of rhetorical strategies:
  - Counter-arguments and rebuttals
  - Appeals to authority, fear, emotion
  - Common logical fallacies (false dilemma, ad hominem, slippery slope)
- Suitable text types:
  - Political speeches
  - Editorials and opinion pieces
  - Parliamentary debates
  - Long-form social media posts
- Limitations:
  - Output is interpretive, not ground truth
  - Argument structure is inferred, not empirically verified

---

## 3. Discourse-Sensitive Topic Modeling

- Semantic clustering of texts beyond bag-of-words models
- Identification of topics that reflect:
  - Frames
  - Narratives
  - Ideological positioning
- Typical workflow:
  - Chunk documents into manageable segments
  - Generate embeddings using an LLM or embedding model
  - Cluster segments using statistical methods
  - Use an LLM to describe clusters in discourse-aware terms
- Useful for:
  - Historical corpora
  - Parliamentary archives
  - Newspaper collections
  - Policy documents

---

## 4. Diachronic Discourse Analysis

- Comparison of discourse patterns across time periods
- Analysis of shifts in:
  - Framing strategies
  - Metaphor usage
  - Moral language
  - Attribution of agency
  - Degree of certainty or hedging
- Common research questions:
  - How discourse changes before and after major events
  - How terminology and framing evolve across decades
- Practical considerations:
  - Avoid feeding entire corpora directly
  - Use sampling, summarization, and controlled comparative prompts

---

## 5. Metaphor and Figurative Language Detection

- Identification of metaphorical and figurative expressions
- Classification of metaphor types:
  - War metaphors
  - Disease metaphors
  - Journey metaphors
  - Market or economic metaphors
- Advantages over classical NLP:
  - Better handling of implicit and creative metaphors
- Caveats:
  - Cultural and historical metaphors require domain expertise
  - Outputs should be validated against human interpretation

---

## 6. Discourse Actor and Voice Analysis

- Analysis of who is speaking and from which position
- Detection of:
  - Institutional vs personal voice
  - Collective voice (“we”, “the people”)
  - Shifts in perspective within a single text
- Identification of agency suppression:
  - Passive constructions
  - Vague responsibility attribution
- Value:
  - Difficult to automate reliably without LLM-based interpretation

---

## 7. Cross-Lingual Discourse Comparison

- Comparison of discourse strategies across languages
- Identification of:
  - Framing differences
  - Rhetorical shifts introduced by translation
  - Language-specific discourse markers
- Best practices:
  - Avoid blind reliance on raw machine translation
  - Ask the LLM to justify translation and framing choices
- Particularly useful for:
  - Multilingual political discourse
  - International media analysis

---

## 8. Corpus Exploration and Hypothesis Generation

- Use of LLMs as exploratory research assistants
- Typical tasks:
  - Suggesting discourse categories
  - Proposing coding schemes
  - Highlighting unexpected patterns
- Appropriate use:
  - Early-stage exploration
  - Supporting qualitative reasoning
- Inappropriate use:
  - Drawing causal conclusions
  - Replacing formal statistical analysis

---

## 9. Teaching and Pedagogical Applications

- Classroom uses:
  - Annotating sample texts
  - Comparing human and LLM-based analyses
  - Prompt engineering as a methodological exercise
  - Critical evaluation of bias and hallucination
- Educational value:
  - Enhances students’ critical discourse awareness
  - Makes methodological limitations explicit rather than hidden

---

## 10. Known Limitations of LLMs in Discourse Analysis (as of 2025)

- LLMs should not be relied upon to:
  - Produce accurate frequency counts without verification
  - Replace close reading and contextual analysis
  - Infer author intent reliably
  - Deliver unbiased or neutral interpretations
- Recommended stance:
  - Treat LLMs as interpretive tools
  - Maintain human oversight and theoretical grounding

## Installing OpenAI library

Principles will be very similar to other AI libraries (Mistral, etc)

Open AI library is nactually currently included in Google Colab but not the latest version.


I would imagine Google would want to include some of their products(that is libraries to access them).
Those are Bard and Gemini (as of December 2023).

* Google did most of the original reasearch on Large Language Models in the last 15 years - but has not capitalized on it
* Instead companies such as OpenAI have overtaken Google in the mind-share and usability
* It is a classical case of Innovators Dilemma - Google was making too much money on their Search and they were afraid to hurt their product, now they have little choice
(lots of analogies elsewhere, for example BMW early electrical car looked very different from standard BMW because they were afraid of hurting their main product, now BMW realized that they have to catch up to companies such as Tesle, etc)

OpenAI is not immune, there are companies, startups such as Mistral from France, Europe that are very promising and may overtake OpenAI.
Competition in this field is high - and OpenAI could use some competition, because OpenAI is not actually open...

So pragmatically for now we will be using OpenAI to do our LLM tasks.

In [3]:
import openai # in 2024 the library was not present but now is
print(f"Open AI version {openai.__version__}")
# compare to version at pypi.org
# https://pypi.org/project/openai/

Open AI version 2.9.0


In [ ]:
# so not needed anymore but you can use it to install later versions
# !pip install --upgrade --force-reinstall openai
# THEN RESTART RUNTIME!

In [2]:
import os # standard Python library for os related tasks
# import openai # so this is openai library we just installed

## Getting OpenAI API key

It used to be that you could get OpenAI key for free at least for a month and you could get some credits. Maybe you still get one month or $10 free credits.

The site for OpenAI Develope page is: https://platform.openai.com/docs/overview

To get key go here: https://platform.openai.com/api-keys

KEY IDEA: Never show API keys in your code or notebooks!!!


In [4]:
from google.colab import userdata # API to access user secrets

# for Google Colab users the best way to save Secrets is to use Secrets storage provided by Google Colab
# let's get secret key by name from Secrets storage: OPENAI_DH

secret = userdata.get("OPENAI_DH")
# how long is secret?
print(f"Secret is {len(secret)} characters long")

Secret is 164 characters long


In [5]:
# alternative to give secret by hand
if not secret:
  from getpass import getpass # standard library into Python
  secret = getpass('Enter the secret API key for OpenAI value: ') # so very similar to input function but with stars...
else:
  print(f"I already have secret key")

# the 3rd option would be to load the secret key from some configuration or .env file that is also fine AS LONG as that file is not publicly visible anywhere :0

I already have secret key


In [6]:

# openai.api_key = "use_your_own_key!"  #never share your private API keys with the world! read them from enviroment or private text file
# or using getpass and copy pasting (like I did - not very convenient but good for one time use)
# alternatives, store API keys on your Google Drive - personally I do not recommend
# there is also something called Google Secrets, again I personally do not trust it, but your mileage may vary
openai.api_key = secret  #never share your private API keys with the world! read them from enviroment or private text file

## Accessing the API

https://github.com/openai/openai-python - documents the changes

Since this is cutting edge research, API changes quite often.

Expect things to stabilize in a few years.

In [8]:
from openai import OpenAI
client = OpenAI(
    # This is the default and can be omitted
    # api_key=os.environ.get("OPENAI_API_KEY"),
    api_key=secret, # instead of accessing my api key from my os, i use the key i pasted in to secret
    # again secret is just a string but do not write it directly here!!
)

In [9]:
chat_completion = client.chat.completions.create(
    # so we pass one or messages to OpenAI endpoint, in this just
    # effectively messages are just a list of dictionaries
    # here a single dictionary representing a single message
    messages=[
        {
            "role": "user",
            "content": "Say this is a test but translate into Latvian", # so this is your prompt
        }
    ],
    model="gpt-5-nano-2025-08-07", # this is among the cheapest of the models similar to the free version on ChatGPT
    # there are more options but we will stick with basics
)
# so with this example we made a call to OpenAI using the API key

In [12]:
# we can get the json response of everything the model provided
# chat_completion.json()
chat_completion.model_dump_json() # so this is the new approach as of 2025

'{"id":"chatcmpl-Co9i2WTJZUohB7EIhok8PyX6Yat47","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"Pateik, ka tas ir tests.","refusal":null,"role":"assistant","annotations":[],"audio":null,"function_call":null,"tool_calls":null}}],"created":1766069758,"model":"gpt-5-nano-2025-08-07","object":"chat.completion","service_tier":"default","system_fingerprint":null,"usage":{"completion_tokens":2770,"prompt_tokens":16,"total_tokens":2786,"completion_tokens_details":{"accepted_prediction_tokens":0,"audio_tokens":0,"reasoning_tokens":2752,"rejected_prediction_tokens":0},"prompt_tokens_details":{"audio_tokens":0,"cached_tokens":0}}}'

In [11]:
# now to get just the answer in text form # so you can see that Latvian is not the greatest here
chat_completion.choices[0].message.content

'Pateik, ka tas ir tests.'

In [13]:
len(chat_completion.choices) # how many choices do we have?

1

## Automating Sentiment Analysis

To avoid writing all the boileplate code by hand, instead I will write a function that combines all the required code and I can call the function instead, whenever I need sentiment analysis.

In [15]:
# so we can predefine our own commonly used settings such as partial prompts, models, how many input symbols to use, etc.
def getSentiment(prompt, client=client, sentiments=("positive","neutral","negative"),
                #  model="gpt-3.5-turbo", # from 2024
                 model="gpt-5-nano-2025-08-07", # cheapest from 2025
                 max_prompt=200 # you can control how many tokens you get in input
                 ):
    sentiment_text = ",".join(sentiments)  # I add all the sentiments in a string separated by comma
    prompt=f"Social media post: \"{prompt[:max_prompt]}\"Sentiment ({sentiment_text}):"
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )
    return chat_completion.choices[0].message.content

In [16]:
getSentiment("I really like bread and circuses")
# depending on temperature setting we might get different answers

'Positive\n\nReason: The post states that the author “really likes” bread and circuses, which expresses a positive sentiment toward the concept. (Context or sarcasm could alter this in some cases.)'

In [17]:
getSentiment("Where do I begin? This is a brand new 4K scan from the original negative of the movie with an added HDR10 & Dolby Vision HDR grading, which looks fantastic!")

'positive'

In [18]:
getSentiment("Man patik alus") # I like beer in Latvian

'positive'

In [19]:
getSentiment("Man nepatīk slidenas ielas") #I do not like slippery streets

'negative'

In [ ]:
# using this custom function you can pass in your own sentiments - for example in Latvian
getSentiment("Man nepatīk slidenas ielas", sentiments=["pozitīvs", "neitrāls", "negatīvs"])

'Negatīvs'

In [20]:
getSentiment("Man patik alus", sentiments=["pozitīvs", "neitrāls", "negatīvs"])

'Pozitīvs. (Izsaka patiku pret alu.)'

## Prompt creation

So basic  idea is we provide a sort of answer key when we need sentiment analysis. We ended our prompt with possible answers and LLM gave us one of them.

In [21]:
# so this is our own ChatGPT basically, except we can adjust more parameters
def getAnswer(prompt, client=client,
              # model="gpt-3.5-turbo",
              model="gpt-5-nano-2025-08-07",
              max_prompt=500):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt[:max_prompt],
            }
        ],
        model=model,
    )
    return chat_completion.choices[0].message.content

In [22]:
getAnswer("What is the capital of Latvia?")

'Riga.'

In [23]:
# of course LLMs have a infamous tendendcy to hallucinate especially when the model does not have the full answer.
# One possible test to evaluate is to ask a question where the answer is between ranges of some information it already knows
getAnswer("Who was president of Latvia in 1926?") # key being that 1926 was a year President of Latvia did not do anything special

'Jānis Čakste. He served as President of Latvia from 1922 until his death in 1927, so in 1926 the president was Čakste.'

In [24]:
getAnswer("Who was president of Latvia in 1926? Write a short paragraph on this president") # key being that 1926 was a year President of Latvia did not do anything special

'Jānis Čakste.\n\nShort paragraph:\nJānis Čakste was the first President of Latvia, serving from 1922 until his death in 1927. A Latvian lawyer and statesman, he played a pivotal role in shaping Latvia’s early independence and constitutional framework, guiding the country through its formative years with a focus on parliamentary democracy and political stability. He remained in office through 1926 and died in 1927, leaving a legacy of constitutional integrity and steady leadership during Latvia’s interwar period.'

### getAnswer as analogous to ChatGPT interface

So now if you had say 100 documents to analyse you could write a loop and pass this function 100 times.

In [25]:
# so there is this idea in prompting LLMs called shots, zero-shot (no examples)
# one shot - single example
# few shot - some examples
def getMovieEmoji(movie_title,client=client,
                  # model="gpt-3.5-turbo"
                  model="gpt-5-nano-2025-08-07"
                  ):
    prompt=f"""Back to Future: 👨👴🚗🕒
    Batman: 🤵🦇
    Transformers: 🚗🤖
    Wonder Woman: 👸🏻👸🏼👸🏽👸🏾👸🏿
    Winnie the Pooh: 🐻🐼🐻
    The Godfather: 👨👩👧🕵🏻‍♂️👲💥
    Game of Thrones: 🏹🗡🗡🏹
    {movie_title}: """
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )
    return chat_completion.choices[0].message.content

In [26]:
print(getMovieEmoji("The Bourne Conspiracy"))

The Bourne Conspiracy: 🕵️‍♂️🧠💥🚗💨🏃‍♂️🔎

Reason: spy/amnesia themes (detective), memory issues (brain), action/explosions, car chases, and pursuit/investigation. Want another variation (more focused on amnesia or CIA)? I can adjust.


In [30]:
# so let's try with 3.5
print(getMovieEmoji("The Bourne Conspiracy", model="gpt-3.5-turbo")) # much quicker, but might not be as relevant

🕵️🕵️‍♂️🔫💥🔪


In [31]:
print(getMovieEmoji("Top Gun: Maverick"))

Nice puzzle. Here are a few emoji-clue options for Top Gun: Maverick. Pick the vibe you prefer (jet/flight emphasis, Navy pilot, speed, etc.):

- Option 1: ✈️👨‍✈️🔥🏁
  - Jet, pilot, intensity, finish/speed vibe (Maverick’s famous need for speed).

- Option 2: 🛩️👨‍✈️🌊🏟️
  - Fighter jet, Navy pilot, carrier/sea setting, action-packed.

- Option 3: ✈️🧑‍✈️💥💨
  - Jet, pilot, explosion/high-adrenaline moments, speed lines.

- Option 4: 🛫👨‍✈️🪖🏁
  - Takeoff, pilot, military deco, finish line (competition/flight goal).

If you want a single, compact set: ✈️👨‍✈️🔥🏁 captures the core idea of Top Gun and Maverick nicely.


In [32]:
print(getMovieEmoji("Frozen 2"))

Nice idea. Here are some Frozen 2 emoji options you can use, focusing on the two sisters, the forest journey, and the elemental spirits.

- Basic: ❄️👭🌲🪄
- With the four elements (fire, water, wind, earth) hinted: ❄️👭🔥💧💨🌲🪨
- Including the journey/kingdom vibe: ❄️👭🏔️🌲🪄💧
- Nokk and forest spirits vibe: ❄️👭🌊🐎🌬️🪄🌲

Want me to tailor it to a specific aspect (Elsa/Anna, the forest, the spirits, or the songs) or make a shorter/longer version?


## Key takeaway - answers are not deterministic

Notice how multiple calls can return different answers, it is due to the temperature setting that slightly changes the answers.

Model stays the same but there is an element of slight randomness present.

So when you see some amazing LLM examples in action you have to ask how many tries did it take to generate them. Case in point recent Google Gemini Demo presentation.

In [34]:
# https://www.esrb.org/
def getGameRating(movie_text,client=client, model="gpt-3.5-turbo"):
    prompt=f""""Provide an ESRB rating for the following text:
    {movie_text}
    ESRB rating:"""
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
        max_tokens=60,
        temperature=0.7 # temperature ranges from 0 to 2
    )
    return chat_completion.choices[0].message.content


In [35]:
getGameRating("The game opens with a screen in a dark and stormy night. Five gamblers sit around a fire in a dark forest.")
# so one of the issues with LLMs is that you need to provide enought context

'Teen - Mild Violence, Simulated Gambling'

In [36]:
def getMovieRating(movie_text,client=client, model="gpt-3.5-turbo"):
    prompt=f""""Provide an MPA rating to the movie based on following description:
    {movie_text}
    MPA rating:"""
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
        max_tokens=60,
        temperature=0.7 # temperature ranges from 0 to 1
    )
    return chat_completion.choices[0].message.content

In [37]:
getMovieRating("They say all happy families are alike but all unhappy families are different in their own way")

'PG-13'

In [38]:
def getStudyNotes(subject, client=client, model="gpt-3.5-turbo"):
    prompt=f"What are some key points I should know when studying {subject} Provide at least five key points\n\n1."
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
        # using default for now, you can play around with some of the parameters
        # temperature=1,
        # max_tokens=64,
        # top_p=1.0,
        # frequency_penalty=0.0,
        # presence_penalty=0.0
    )
    return chat_completion.choices[0].message.content

In [39]:
print(getStudyNotes("Riga")) # so this is from 3.5

Riga is the capital and largest city of Latvia, located on the Baltic Sea coast.
2. The city's historic center, known as Old Riga, is a UNESCO World Heritage Site and is home to well-preserved medieval architecture including cathedrals, churches, and charming cobblestone streets.
3. Riga is known for its vibrant cultural scene, with numerous museums, art galleries, theaters, and music venues showcasing the city's rich history and contemporary creativity.
4. The city is a popular destination for tourists, offering a wide range of attractions such as the Riga Central Market, the Latvian National Opera, and the Art Nouveau district filled with stunning architectural masterpieces.
5. Riga is also a hub for business and commerce in the Baltic region, with a growing economy and a diverse range of industries including finance, technology, and logistics.


In [40]:
# now let's try it with latest and greatest 5.2 - came out in December 2025
print(getStudyNotes("Riga", model="gpt-5.2"))# this is the full version not NANO

1. **Geography & setting**  
   - Riga is the **capital of Latvia**, located on the **Daugava River** near the **Gulf of Riga (Baltic Sea)**. Its position made it a major **trade and port city**.

2. **Historical development**  
   - Founded in **1201**, Riga grew as an important city in the **Hanseatic League**, connecting Baltic trade routes with Northern Europe.

3. **Cultural and political influences**  
   - Riga’s history reflects shifting rule and influence from **Germanic elites**, **Sweden**, the **Russian Empire**, and later the **Soviet Union**, shaping its language politics, institutions, and demographics.

4. **Architecture & UNESCO significance**  
   - Riga’s historic center is a **UNESCO World Heritage Site**, known for its **Art Nouveau (Jugendstil)** architecture—one of the largest and best-preserved collections in Europe—alongside medieval Old Town buildings.

5. **Modern Latvia: economy and society**  
   - Today Riga is Latvia’s **largest city and economic hub**, c

In [41]:
# i am using print because this answer includes newlines
study_notes_on_dd = getStudyNotes("Digital Discourse")
print(study_notes_on_dd)
# note how 1. is missing from the answer?
# why?
# because I already provide 1. at the end of my prompt
# remember LLMs are just word prediction machines (with some useful side effects)

Digital discourse refers to communication practices that take place online, such as social media interactions, email exchanges, and online forums.

2. The tone and language used in digital discourse can vary greatly depending on the platform and the relationship between the communicators. It is important to understand the context in which communication is taking place in order to interpret it correctly.

3. Digital discourse can be influenced by factors such as anonymity, immediacy, and the ability to communicate with a large audience. These factors can impact the way people communicate and the content of their messages.

4. Understanding digital discourse can help individuals analyze and critically evaluate online communication, including identifying bias, misinformation, and manipulation tactics used in digital spaces.

5. Studying digital discourse can also help individuals improve their own online communication skills, such as writing clear and effective messages, engaging with oth

In [42]:
# now i can save my notes to text file
with open("study_notes.txt", mode="w", encoding="utf-8") as f:
    f.write(study_notes_on_dd)

In [43]:
# let's do the same with 5.2
# let's generate nots on 5.2
study_notes_on_dd_52 = getStudyNotes("Digital Discourse", model="gpt-5.2")
print(study_notes_on_dd_52)

1. **Digital discourse is shaped by the platform.**  
   Each platform’s features (character limits, threading, “likes,” hashtags, algorithmic feeds, moderation tools) influence what people say, how they say it, and how messages spread.

2. **Meaning is often multimodal, not just textual.**  
   Digital communication commonly combines text with images, GIFs, emojis, audio, video, formatting, and links—so you study how these modes work together to create meaning.

3. **Context and audience are fluid (“context collapse”).**  
   Online messages can reach multiple audiences at once (friends, employers, strangers), and can be reshared beyond the original setting—changing interpretation and raising risks around privacy and reputation.

4. **Identity and stance are actively performed.**  
   Users construct identities through usernames, profile elements, language style, memes, and interaction patterns. You’ll often analyze how people signal tone, credibility, expertise, humor, or alignment w

## Conclusion on use of LLMs in Digital Discourse

So LLMs can aid in discourse analysis with what:

* Automated Text Analysis
* Sentiment Analysis
* Topic Modeling
* Translation
* Context Analysis
* Cross-cultural Analyss
* Summarization
* Bias detection

In [ ]:
notes = [getStudyNotes("Sentiment Analysis") for _ in range(5)]  # i ran the same query 5 times, so text completion will be different each tie
print(notes)

[' Keywords and phrases can be negative even when they don’t seem to be\n2. Popular sentiment analysis is not always the right analysis\n3. Slang and capitals can make sentences seem negative', ' Semantria tracks sentiment globally by applying sentiment analysis models to content expressed in multiple languages.\n2. Semantria sentiment analysis models come in three different methods which are Query Based, Feed Based, and Sentiment Match.\n3. With Query Based sentiment analysis you can input a phrase or sentence for Semantria', " Find out what is interesting about your business\n2. Focus on your buying process\n\n3. Reduce capital inventory\n4. Have the temperament of an artists\n\nWhat key points do you think I should know for this topic?\n\nYou shouldn't study how to do sentiment analysis or focus on this topic if", ' Authors generally use sentiment words to convey approval versus a negative sentiment\n a group holds about a person, company, issue, or life events.\n2. Automated sentim

In [ ]:
digital_discourse_notes = [getStudyNotes("Digital Discourse") for _ in range(5)]
for note in digital_discourse_notes:
  print(note)
  print("="*40)

 There is a relationship between the word used and the identity of the individual speaker.

2. Gender can also change depending on the audience.

3. Tone serves as a crucial component for comprehension.

4. Slang and jargon differ from culture to culture and language to language

5.
Doesn't exist in a bubble so we need to pay attention to the framing and the understanding and assumptions around the discourse

2.It can be participatory and evolving

3.We need to study the social and political dimensions

4.Water cooler talk

5. A means of both self
 Digital Discourse includes the many locations of the discussion, such as a blog, a message board, a chatroom, a wiki, a MySpace page, a Facebook page, a YouTube video, etc.
A) Different people have different digital practices when it comes to culture

2. There are "four
 Textual and Visual 2. Readers and Writers 3. Poetic Knowledge and Poetic Practice 4. Technical and Cultural 5. Historical and Scientific 6. Analysis and Interdisciplinarity


In [ ]:
def getEssayOutline(subject):
  response = openai.Completion.create(
  engine="davinci",
  prompt=f"Create an outline for an essay about {subject}:\n\nI: Introduction",
  temperature=0.7,
  max_tokens=60,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)
  return response["choices"][0]["text"]

In [ ]:
print(getEssayOutline("Julius Cesar"))



II: Julius Cesar

III: Family background

IIII: Early life

IIII: Civil service

V: Cesar and Crassus

VI: Cesar and Pompey

VII: Cesar and the provinces

VIII:


In [ ]:
print(getEssayOutline("Tourism in Latvia"))

.

- Tourism in Latvia.

II: What are the main development directions of the tourism business in Latvia?

- Growth of the tourism industry.

- The tourism industry as one of the most dynamic economic sectors in Latvia.

- Importance of the tourism industry


In [ ]:
def getHorrorStory(topic):
  response = openai.Completion.create(
  engine="davinci",
  prompt=f"Topic: Breakfast\nTwo-Sentence Horror Story: He always stops crying when I pour the milk on his cereal. I just have to remember not to let him see his face on the carton.\n###\nTopic: {topic}\nTwo-Sentence Horror Story:",
  temperature=0.5,
  max_tokens=60,
  top_p=1.0,
  frequency_penalty=0.5,
  presence_penalty=0.0,
  stop=["###"]
)
  return response["choices"][0]["text"]

In [ ]:
print(getHorrorStory("snow"))

 I was walking home from work when I realized that I was the only one on the sidewalk. A few minutes later, I saw a snowplow coming down the road. I waved to get its attention, but it just kept on going.
I don't know what happened to everyone else.


In [ ]:
print(getHorrorStory("Christmas"))

 The real Santa Claus was too fat to fit down the chimney, so he left a note saying he'd be back next year.
Two-Sentence Horror Story: I think Santa Claus is going to kill me.



In [ ]:
# once I have a list of some texts to analyzie
# i can simply loop over them and clal my getSentiment function for each text/document
my_tweets = ["In October main exports partners were Lithuania, Estonia, Germany and United Kingdom. The main import partners were Lithuania, Russian Federation, Poland and Germany."
,"In October 2021 the foreign trade turnover of Latvia amounted to € 3.39 billion, which at current prices was 23.5% larger than a year ago.Exports value of goods was ⬆️17.8% higher, but imports value of goods ⬆️28.8% higher. "]

my_tweet_sentiments = [getSentiment(tweet) for tweet in my_tweets]
my_tweet_sentiments

[' Positive', ' 0']

## Alternatives to OpenAI API

These days you have many many alternative LLM providers, such as Gemini from Google, Claude from Anthropic, Mistral, DeepSeek and many more

You could apply for API keys at individual ones, but my recommendation is to use an aggregrator.

I personally use https://openrouter.ai which let's me use SAME API to access pretty much any Large Language Model in the world - at least 400 as of last count.

Downside you pay about 5% premium for this convenience.

Sometimes this is actually unavoidable if you want to connect to some APIs as some providers are very hard to get a paid account and set it up, even Google makes things too difficult at times.

Another advantage of Openrouter.ai API keys is that you can set a SPENDING LIMIT on how much you want that key to allow.

This is crucial if you are worried of letting some script run wild and rack up huge bills. :)



## Premade Jupyter Notebook for Batch processing

If you want to avoid coding your own notebook, you can use

While working at various projects at National Library of Latvia, I've had colleagues ask for batch processing notebook:

You are also free to use it - just use your own openrouter.ai key:

https://colab.research.google.com/github/LNB-DH/PublicTools/blob/main/notebooks/llm/batch_processing.ipynb

